# Homework 3

### By James Kinney and Franklin Marsh

MATH0154
with Prof. 😁Gabe Chandler😁 

First, we need to generate $x_{0}$ which is the initial sequence of numbers $[1,2,3...,m]$. The following function generates $x_{0}$ for arbitrary $m$.

🙌 👏 🍻

In [7]:
GenX0 <- function(m = NULL, initialPerm = NULL) {
    #
    # Function to generate the sequence of the first m positive integers [1,2,3...m]
    #
    # Args:
    #    m: integer length of the sequence
    #    initialPerm: a vector of numbers to be sorted
    # Returns:
    #     x0: sequence integers: [1,2,3...m]
    
    
    # The input should be either an inital permutation of an m
    # If both are given, the m will be used
    if (!is.null(m)){
        return(c(1:m))
    }
    # If m is null, sort the initial vector into increasing order
    else {
        return (sort(initialPerm, decreasing = FALSE))
    }
}


In [8]:
ex_sequence <- GenX0(7) #example sequence
print(ex_sequence) #print the example sequence

[1] 1 2 3 4 5 6 7


To generate a list of vectors that could possibly be neighbors, we need to swap two elements in the initial vector $x_{0}$. For this, we will write a swap function.

In [9]:
Swap <- function (x, i, j) {
    #
    # Function to swap two elements (i,j) in an input vector x.
    #
    # Args:
    #    x: sequence on which to perform the swap.
    #    i: index less than length(x)
    #    j: index less than length(x)
    # Returns:
    #     x_swap: new vector with indices i and j swapped.
    
    tempI <- x[i] # Save the ith element
    x[i] <- x[j] # Change the ith element to the jth element
    x[j] <- tempI # Change the jth element to the ith element
    return (x) # Return the new vector
}


In [10]:
ex_swap <- Swap(ex_sequence, 2, 4) #swap the 2nd element with the 4th element.
print(ex_swap) #print the sequence with the 2nd element and the 4th element swapped.

[1] 1 4 3 2 5 6 7


We will now write a function to generate all of the possible two-element swaps in a vector.

In [11]:
GenSwaps <- function(x) {
    #
    # Function to generate all unique swaps of  two elements (i,j) in an input vector x.
    #
    # Args:
    #    x: sequence on which to perform the generation of the list of 1 possilbe swap.
    # Returns:
    #    output: vector of possible swaps
    # Dependencies:
    #    function: Swap
    
    # Input is a vector of the permutation in order

    output <- c()   # Initialize empty list to hold all possible permutations with 1 swap
    for (i in (1:length(x))) { # Loop through each element and generate the swaps with all the elements after it
        for (j in (i:length(x))) {
        append(output, Swap(x, i, j), after = length(output)) # Add each new potential neighbor to the vector to store them
        }
    }
    return(output)
}

In [14]:
ex_gen_swaps <- GenSwaps(ex_sequence)
print(ex_gen_swaps)

NULL


We will need to find the size of every possible swap created by GenSwaps. The size of a vector $V$ is defined as: $\sum_{i}^{m}i \times V_{i}$, where $m$ is the length of the vector $V$.

In [17]:
FindSize <- function(x) {
    #
    # Function to check the size of a permutation
    #
    # Args:
    #    x: permutation to find the size of
    # Returns:
    #    output: vector of possible swaps
    # Dependencies:
    #    function: Swap
  # Input is a vector of the permutation in order
  sizes <- c()
  # Loop through each index of the permutation and multiply by the index
  for (j in (1:length(x))) {
    # Store these values in a vector
    sizes[j] <- x[j]*j
  }
  # Return the sum of the vector of products
  return(sum(sizes))
}

In [ ]:
# A function which returns TRUE if the algrithm says to switch and FALSE if not
ChangeCheck <- function (nX, nY) {
  # Inputs are n(x) and n(y) - the number of neighbors of the start x and the
  # proposed neighbor to switch to y
  # If n(x) is greater than n(Y) always switch
  if (nX >= nY) {
    return (TRUE)
  }
  # Otherwise switch with probability n(x)/n(y)
  else {
    # Create a unif between 0 and 1 and if it is less than n(x)/n(y) switch
    p <- nX/nY
    u <- runif(1, 0, 1)
    if (u <= p) {
      return (TRUE)
    }
    else {
      return (FALSE)
    }
  }
}

Out of a vector of neighbors, we need to pick one at random.
We can use the R function "sample" for this, or we can use the following:

In [ ]:
# A function to pick a random neighbor
RandomNeighbor <- function (neighbors) {
  # Input is a vector of the neighbors
  index <- runif (1, 1, length(neighbors))
  return (neighbors[index])
}

In [ ]:
# A function to find which of the possible swaps are neighbors when given the swaps and the k size cutoff
WhichNeighbors <- function (possibleSwaps, k) {
  neighbors <- c()
  # Loop through each swap checking if its size is greater than k
  for (j in 1:length(possibleSwaps)) {
    if (FindSize(possibleSwaps[j]) >= k){
      # If the swap produces a neighbor store it in the neighbors vector
      append(neighbors, possibleSwaps[j], after = length(neighbors))
    }
  }
  return (neighbors)
}


In [3]:
# A markov chain monte carlo algorithm to find the average size of permutations
# whose size is greater than a given k
MCMC <- function (m = NULL, initPerm = NULL, k) {
  # The input should be either an inital permutation or an m implying use the first m natural numbers
  # and a k which is the lower cutoff for "large" permutations
  # Start by creating a vector to store the mean of the sizes of large perm'ns
  # Also store the number that have been added so we can calc avg size
  bigPermAvg <- c(NULL, 0)
  # Generate the first permutation (the largest one)
  x <- GenX0(m, initPerm)
  # Confirm its size is greater than k
  if (FindSize(x) < k) {
    return (NULL)
  }
  # Store the size of x0 and update the number in bigPermAvg[2]
  bigPermAvg[1] <- FindSize(x)
  bigPermAvg[2] <- bigPermAvg[2] + 1
  # Designate the number of iterations to loop through
  n <- 1000
    for (i in 1:n) {
      # Generate all possible neighbors of x
      xSwaps <- GenSwaps(x)
      # Find which of the possible neighbors actually are neighbors (i.e. size >= k)
      xNeighbors <- WhichNeighbors(xSwaps, k)
      # Loop through possible y permutations until we find one to switch to
      swapped <- FALSE
      while (!swapped){
        # Pick a random neighbor to possibly switch to
        y <- RandomNeighbor(xNeighbors)
        # Generate the neighbors of y
        ySwaps <- GenSwaps(y)
        yNeighbors <- WhichNeighbors(ySwaps, k)
        # If n(x) > n(y) or we choose to switch with prob n(x)/n(y) switch to the y permutation
        if (ChangeCheck(length(xNeighbors), length(yNeighbors))) {
          x <- y
          swapped <- TRUE
        }
      }
      # Update the average big size and the number of perm'ns that have been used
      bigPermAvg[2] <- bigPermAvg[2] + 1
      c <- bigPermAvg[2]
      bigPermAvg[1] <- bigPermAvg[1]*((c-1)/c)+(FindSize(x)/c)
    }
  # Find the average size of all of the big permutations we've visited and return this value
  return (bigPermAvg)
}